In [7]:
import json
import pandas as pd

from config import json_datapath, json_graphpath, csv_graphpath

In [8]:
with open(json_datapath, 'r') as json_file:
    data = json.load(json_file)
    dao_ids = {
        dao['UID']: dao['Project Name']
        for dao in data
    }
    for dao in data:
        friends = list(set([d for d in dao['Friend IDs'] if d in dao_ids.keys()]))
        dao.update({
            "DAO Friend IDs": friends,
            "DAO Friends": [dao_ids.get(f) for f in friends]
        })
    for dao in data:
        followers = []
        for d in data:
            if dao['UID'] in d['DAO Friend IDs']:
                followers.append(d['UID'])
        dao.update({
            "DAO Follower IDs": followers,
            "DAO Followers": [dao_ids.get(f) for f in followers]
        })

In [9]:
df = pd.DataFrame(data)
df[['UID', 'Project Name', 'Twitter', 
    'DAO Friend IDs', 'DAO Friends', 
    'DAO Follower IDs', 'DAO Followers'
   ]].to_json(json_graphpath, orient='records', indent=4)

df['Logo URL'] = df['Meta Data'].apply(lambda x: x['profile_image_url'])
df['DAO Friends'] = df['DAO Friends'].str.join(', ')
df['DAO Followers'] = df['DAO Followers'].str.join(', ')

df[['UID', 'Project Name', 'Twitter', 
    'Logo URL', 'DAO Friends', 'DAO Followers'
   ]].to_csv(csv_graphpath)

df.head()

,UID,Project Name,Twitter,Updated,Meta Data,Friend IDs,Follower IDs,DAO Friend IDs,DAO Friends,DAO Follower IDs,DAO Followers,Logo URL
0,6135982,1729,oneseventwonine,20220308-13-00,"{'id': 6135982, 'id_str': '6135982', 'name': '...",[],[],[],,"[1261307955464593410, 3184528398, 17189394]","Ants-review, Artizen, Nader Dabit",http://pbs.twimg.com/profile_images/1375067996...
1,1370576489909993472,_slow_crypto - Fabrics of Reality,_slow_crypto,20220308-13-00,"{'id': 1370576489909993472, 'id_str': '1370576...","[1430431386280804357, 50883209, 81748683336176...",[],"[1450577818656538624, 996552749893304320, 1436...","Surge, 1Hive, Dream DAO, SuperBenefit, Leyline...",[],,http://pbs.twimg.com/profile_images/1491209412...
2,1475869555826253828,0xdaoApe,0xdaoape,20220308-13-00,"{'id': 1475869555826253828, 'id_str': '1475869...","[2827560493, 1593887875, 80394870, 11620016259...",[],"[996552749893304320, 1316549542041448448, 1105...","1Hive, TEC, The Commons Stack, GitcoinDAO, The...","[798810763586478080, 1316549542041448448]","Giveth, TEC",http://pbs.twimg.com/profile_images/1475869715...
3,996552749893304320,1Hive,1hiveorg,20220308-13-00,"{'id': 996552749893304320, 'id_str': '99655274...","[1461377873323900935, 1368846018314403842, 142...",[],"[1028536069330022400, 1105766889521860608, 856...","Curve Labs, The Commons Stack, GitcoinDAO, The...","[1370576489909993472, 1475869555826253828, 142...","_slow_crypto - Fabrics of Reality, 0xdaoApe, 1...",http://pbs.twimg.com/profile_images/1276286557...
4,1421237328719360010,1Hive Gardens,gardensdao,20220308-13-00,"{'id': 1421237328719360010, 'id_str': '1421237...","[1310159840652779520, 914738730740715521, 1117...",[],"[1316549542041448448, 798810763586478080, 9965...","TEC, Giveth, 1Hive","[1475869555826253828, 996552749893304320, 1391...","0xdaoApe, 1Hive, Citizen DAO, Giveth, Longtail...",http://pbs.twimg.com/profile_images/1479201973...


In [12]:
from community import community_louvain
import networkx as nx

In [21]:
df = pd.DataFrame(data)
G = nx.from_dict_of_lists(df.set_index('Project Name')['DAO Friends'].to_dict())
G.remove_edges_from(nx.selfloop_edges(G))
G = nx.k_core(G, 2)
G.number_of_nodes()

165

In [23]:
degrees = pd.Series(dict(G.degree), name='Connections')
partition = pd.Series(community_louvain.best_partition(G), name="Group")
combined = pd.concat([degrees, partition], axis=1).reset_index()
combined.rename(columns={'index': 'Project'}, inplace=True)
combined

,Project,Connections,Group
0,1729,3,4
1,_slow_crypto - Fabrics of Reality,7,1
2,0xdaoApe,8,2
3,1Hive,20,2
4,1Hive Gardens,12,2
...,...,...,...
160,VitaDAO,36,0
161,We3,3,4
162,Web3 University,4,4
163,web3baddies,14,4


In [28]:
pd.DataFrame(G.edges(), columns=['source', 'target']).to_csv('../data/flourish_links.csv')

In [29]:
combined.to_csv('../data/flourish_points.csv')